In [5]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import requests
import json

In [6]:
# Read accident CSV file as Pandas DataFrame
accident_df = pd.read_csv(r"accident_FARS.csv", encoding='cp1252')
accident_df

C:\Users\shrey\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (40,42,69) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,STATE,STATENAME,ST_CASE,VE_TOTAL,VE_FORMS,PVH_INVL,PEDS,PERSONS,PERMVIT,PERNOTMVIT,...,HOSP_MN,HOSP_MNNAME,CF1,CF1NAME,CF2,CF2NAME,CF3,CF3NAME,FATALS,DRUNK_DR
0,1,Alabama,10001,2,2,0,0,3,3,0,...,27,27,0,None,0,None,0,None,1,1
1,1,Alabama,10002,2,2,0,0,2,2,0,...,99,Unknown EMS Hospital Arrival Time,0,None,0,None,0,None,1,0
2,1,Alabama,10003,3,3,0,0,4,4,0,...,5,5,14,"Motor Vehicle struck by falling cargo,or somet...",0,None,0,None,1,0
3,1,Alabama,10004,1,1,0,1,1,1,1,...,88,Not Applicable (Not Transported),0,None,0,None,0,None,1,0
4,1,Alabama,10005,1,1,0,0,1,1,0,...,88,Not Applicable (Not Transported),0,None,0,None,0,None,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33239,56,Wyoming,560117,1,1,0,0,1,1,0,...,99,Unknown EMS Hospital Arrival Time,0,None,0,None,0,None,1,0
33240,56,Wyoming,560118,5,5,0,0,6,6,0,...,0,0,0,None,0,None,0,None,1,0
33241,56,Wyoming,560119,1,1,0,0,1,1,0,...,21,21,0,None,0,None,0,None,1,1
33242,56,Wyoming,560120,1,1,0,0,2,2,0,...,52,52,0,None,0,None,0,None,1,1


In [7]:
# Drop unnecessary columns
accident_df.drop(columns = ["STATE", "CITY", "VE_TOTAL", "VE_FORMS", "PVH_INVL", "PEDS", "PERMVIT", 
                                "PERNOTMVIT", "COUNTY", "CITY", "DAY", "DAYNAME", "MONTH", "YEAR", "DAY_WEEK",
                                "HOUR", "HOURNAME", "MINUTE", "MINUTENAME", "NHS", "NHSNAME", "ROUTE",
                                "TWAY_ID", "TWAY_ID2", "RUR_URB", "FUNC_SYS", "RD_OWNER", "RD_OWNERNAME",
                                "MILEPT", "MILEPTNAME", "LATITUDENAME", "LONGITUDNAME", "SP_JUR", "SP_JURNAME",
                                "HARM_EV", "MAN_COLL", "RELJCT1", "RELJCT2", "RELJCT1NAME", "RELJCT2NAME",
                                "TYP_INT", "TYP_INTNAME", "WRK_ZONE", "WRK_ZONENAME", "REL_ROAD", "REL_ROADNAME",
                                "LGT_COND", "WEATHER1", "WEATHER1NAME", "WEATHER2", "WEATHER2NAME",
                                "WEATHER", "SCH_BUS", "SCH_BUSNAME", "RAIL", "RAILNAME", "NOT_HOUR", 
                                "NOT_HOURNAME", "NOT_MIN", "NOT_MINNAME", "ARR_HOUR", "ARR_HOURNAME",
                                "ARR_MIN", "ARR_MINNAME", "HOSP_HR", "HOSP_HRNAME", "HOSP_MN", "HOSP_MNNAME",
                                "CF1", "CF1NAME", "CF2", "CF2NAME", "CF3", "CF3NAME", "FATALS", "FUNC_SYSNAME", 
                                "DRUNK_DR"], inplace = True)

In [8]:
# Display the dataframe
accident_df

,STATENAME,ST_CASE,PERSONS,COUNTYNAME,CITYNAME,MONTHNAME,DAY_WEEKNAME,ROUTENAME,RUR_URBNAME,LATITUDE,LONGITUD,HARM_EVNAME,MAN_COLLNAME,LGT_CONDNAME,WEATHERNAME
0,Alabama,10001,3,LEE (81),OPELIKA,February,Thursday,Interstate,Urban,32.666222,-85.336658,Motor Vehicle In-Transport,Front-to-Rear,Daylight,Clear
1,Alabama,10002,2,ETOWAH (55),GADSDEN,January,Wednesday,Interstate,Urban,33.997828,-86.053997,Motor Vehicle In-Transport,Front-to-Rear,Dark - Not Lighted,Rain
2,Alabama,10003,4,CLEBURNE (29),NOT APPLICABLE,January,Tuesday,Interstate,Rural,33.660842,-85.391011,Motor Vehicle In-Transport,Front-to-Rear,Dark - Not Lighted,Cloudy
3,Alabama,10004,1,ETOWAH (55),RAINBOW CITY,January,Tuesday,Interstate,Rural,33.956472,-86.140522,Pedestrian,The First Harmful Event was Not a Collision wi...,Dark - Not Lighted,Clear
4,Alabama,10005,1,BALDWIN (3),NOT APPLICABLE,January,Friday,Interstate,Urban,30.656269,-87.809461,Rollover/Overturn,The First Harmful Event was Not a Collision wi...,Dark - Not Lighted,"Fog, Smog, Smoke"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33239,Wyoming,560117,1,CROOK (11),Other,September,Saturday,Unknown,Rural,44.354406,-104.810853,Rollover/Overturn,The First Harmful Event was Not a Collision wi...,Dusk,Severe Crosswinds
33240,Wyoming,560118,6,ALBANY (1),NOT APPLICABLE,September,Monday,State Highway,Rural,41.167136,-105.887706,Motor Vehicle In-Transport,Front-to-Rear,Daylight,Clear
33241,Wyoming,560119,1,SHERIDAN (33),NOT APPLICABLE,September,Sunday,State Highway,Rural,44.912142,-107.104503,Rollover/Overturn,The First Harmful Event was Not a Collision wi...,Daylight,Clear
33242,Wyoming,560120,2,LARAMIE (21),NOT APPLICABLE,September,Sunday,U.S. Highway,Rural,41.403750,-104.456917,Embankment,The First Harmful Event was Not a Collision wi...,Dark - Not Lighted,Clear


In [9]:
# Clean the dataframe
clean_df = accident_df[~accident_df.isin(["NOT APPLICABLE", "Not Reported", "Unknown", "Other"]).any(axis=1)]
clean_df.head(5)

,STATENAME,ST_CASE,PERSONS,COUNTYNAME,CITYNAME,MONTHNAME,DAY_WEEKNAME,ROUTENAME,RUR_URBNAME,LATITUDE,LONGITUD,HARM_EVNAME,MAN_COLLNAME,LGT_CONDNAME,WEATHERNAME
0,Alabama,10001,3,LEE (81),OPELIKA,February,Thursday,Interstate,Urban,32.666222,-85.336658,Motor Vehicle In-Transport,Front-to-Rear,Daylight,Clear
1,Alabama,10002,2,ETOWAH (55),GADSDEN,January,Wednesday,Interstate,Urban,33.997828,-86.053997,Motor Vehicle In-Transport,Front-to-Rear,Dark - Not Lighted,Rain
3,Alabama,10004,1,ETOWAH (55),RAINBOW CITY,January,Tuesday,Interstate,Rural,33.956472,-86.140522,Pedestrian,The First Harmful Event was Not a Collision wi...,Dark - Not Lighted,Clear
7,Alabama,10008,1,HOUSTON (69),COTTONWOOD,February,Saturday,County Road,Rural,31.068686,-85.333658,Rollover/Overturn,The First Harmful Event was Not a Collision wi...,Dark - Not Lighted,Clear
8,Alabama,10009,1,BALDWIN (3),GULF SHORES,February,Friday,Local Street - Municipality,Urban,30.279125,-87.677108,Immersion or Partial Immersion,The First Harmful Event was Not a Collision wi...,Daylight,Clear


In [10]:
# Renaming column names
accidents_df = clean_df.rename(columns={"STATENAME": "State", "ST_CASE": "Case_Number", "PERSONS": "Persons",
                                      "COUNTYNAME": "County", "CITYNAME": "City", "MONTHNAME": "Month",
                                      "DAY_WEEKNAME": "Day", "ROUTENAME": "Route", "RUR_URBNAME": "Rural/Urban",
                                      "LATITUDE": "Latitude", "LONGITUD": "Longitude", "HARM_EVNAME": "First Harmful Event",
                                      "MAN_COLLNAME": "Manner of Collision", "LGT_CONDNAME": "Light Conditions",
                                      "WEATHERNAME": "Weather"})
accidents_df

,State,Case_Number,Persons,County,City,Month,Day,Route,Rural/Urban,Latitude,Longitude,First Harmful Event,Manner of Collision,Light Conditions,Weather
0,Alabama,10001,3,LEE (81),OPELIKA,February,Thursday,Interstate,Urban,32.666222,-85.336658,Motor Vehicle In-Transport,Front-to-Rear,Daylight,Clear
1,Alabama,10002,2,ETOWAH (55),GADSDEN,January,Wednesday,Interstate,Urban,33.997828,-86.053997,Motor Vehicle In-Transport,Front-to-Rear,Dark - Not Lighted,Rain
3,Alabama,10004,1,ETOWAH (55),RAINBOW CITY,January,Tuesday,Interstate,Rural,33.956472,-86.140522,Pedestrian,The First Harmful Event was Not a Collision wi...,Dark - Not Lighted,Clear
7,Alabama,10008,1,HOUSTON (69),COTTONWOOD,February,Saturday,County Road,Rural,31.068686,-85.333658,Rollover/Overturn,The First Harmful Event was Not a Collision wi...,Dark - Not Lighted,Clear
8,Alabama,10009,1,BALDWIN (3),GULF SHORES,February,Friday,Local Street - Municipality,Urban,30.279125,-87.677108,Immersion or Partial Immersion,The First Harmful Event was Not a Collision wi...,Daylight,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33197,Wyoming,560074,2,NATRONA (25),CASPER,August,Monday,Local Street - Municipality,Urban,42.819172,-106.374953,Pedestrian,The First Harmful Event was Not a Collision wi...,Dark - Lighted,Clear
33201,Wyoming,560078,2,SHERIDAN (33),SHERIDAN,August,Wednesday,Interstate,Urban,44.744097,-106.923181,Rollover/Overturn,The First Harmful Event was Not a Collision wi...,Daylight,Clear
33203,Wyoming,560080,1,LARAMIE (21),CHEYENNE,July,Tuesday,Local Street - Municipality,Urban,41.159458,-104.720489,"Post, Pole or Other Supports",The First Harmful Event was Not a Collision wi...,Dark - Not Lighted,Clear
33212,Wyoming,560089,2,FREMONT (13),RIVERTON,May,Saturday,U.S. Highway,Urban,43.047178,-108.458317,Rollover/Overturn,The First Harmful Event was Not a Collision wi...,Dawn,Clear


In [11]:
accidents_df.dtypes

State                   object
Case_Number              int64
Persons                  int64
County                  object
City                    object
Month                   object
Day                     object
Route                   object
Rural/Urban             object
Latitude               float64
Longitude              float64
First Harmful Event     object
Manner of Collision     object
Light Conditions        object
Weather                 object
dtype: object

In [12]:
# Writing Accidents final dataframe to a csv
accidents_df.to_csv(r"Output/accident_final.csv", index=False, header=True)